In [1]:
import pandas as pd
import ast
import re
import nltk
from nltk.corpus import stopwords, words
from nltk import pos_tag

In [2]:
def parse_musixmatch(path):    
    d = {
        'track_id': [],
        'word_counts': []
        }

    with open(path, 'r', encoding='utf-8') as file:
        top_words = None

        # line_count = 0
        for line in file:
            # if line_count >= 25:
            #     break
            # line_count += 1

            line = line.strip()

            if line.startswith('#'):
                continue

            elif line.startswith('%'):
                top_words = line[1:].split(',')
                continue
            else:
                parts = line.split(',')
                track_id = parts[0]
                
                word_counts = {}
                for part in parts[2:]:
                    match = re.match(r'(\d+):(\d+)', part)
                    if match:
                        word = top_words[int(match.group(1)) - 1]
                        count = int(match.group(2))
                        word_counts[word] = count

                d['track_id'].append(track_id)
                d['word_counts'].append(word_counts)

    df = pd.DataFrame(d)
    return df

In [7]:
dataset_path = 'data/mxm_dataset_train.txt'
df = parse_musixmatch(dataset_path)
df

,track_id,word_counts
0,TRAAAAV128F421A322,"{'i': 6, 'the': 4, 'you': 2, 'to': 2, 'and': 5..."
1,TRAAABD128F429CF47,"{'i': 10, 'you': 17, 'to': 8, 'and': 2, 'a': 2..."
2,TRAAAED128E0783FAB,"{'i': 28, 'the': 15, 'you': 2, 'to': 12, 'and'..."
3,TRAAAEF128F4273421,"{'i': 5, 'the': 4, 'you': 3, 'to': 2, 'and': 1..."
4,TRAAAEW128F42930C0,"{'i': 4, 'to': 5, 'and': 7, 'a': 2, 'me': 4, '..."
...,...,...
210514,TRZZZWS128F429CF87,"{'a': 1, 'no': 9, 'que': 7, 'de': 1, 'y': 4, '..."
210515,TRZZZXA128F428ED56,"{'i': 1, 'the': 13, 'you': 6, 'to': 5, 'and': ..."
210516,TRZZZXV128F4289747,"{'i': 13, 'the': 3, 'you': 17, 'to': 5, 'and':..."
210517,TRZZZYV128F92E996D,"{'i': 10, 'the': 6, 'you': 20, 'and': 2, 'me':..."


In [4]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mfard\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping tokenizers\punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mfard\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\mfard\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger_eng.zip.


True

In [8]:
progress = 1

def filter_stop_interjunction(word_counts, total_rows):
    global progress
    print(f"Progress: {(progress * 100 / total_rows):.2f}% → working on {progress} of {total_rows}", end='\r', flush=True)
    progress += 1

    stop_words = set(stopwords.words('english'))

    filtered = {}
    for word, count in word_counts.items():
        if word.lower() not in stop_words:
            tagged = pos_tag([word])
            if tagged[0][1] != 'UH':
                filtered[word] = count

    return filtered      


In [9]:
df['filtered_word_counts'] = df['word_counts'].apply(filter_stop_interjunction, total_rows=len(df))
df

,track_id,word_counts,filtered_word_counts
0,TRAAAAV128F421A322,"{'i': 6, 'the': 4, 'you': 2, 'to': 2, 'and': 5...","{'like': 2, 'de': 1, 'got': 1, 'would': 1, 'se..."
1,TRAAABD128F429CF47,"{'i': 10, 'you': 17, 'to': 8, 'and': 2, 'a': 2...","{'know': 5, 'time': 3, 'la': 7, 'get': 2, 'got..."
2,TRAAAED128E0783FAB,"{'i': 28, 'the': 15, 'you': 2, 'to': 12, 'and'...","{'love': 11, 'like': 1, 'time': 6, 'come': 4, ..."
3,TRAAAEF128F4273421,"{'i': 5, 'the': 4, 'you': 3, 'to': 2, 'and': 1...","{'know': 1, 'got': 3, 'feel': 1, 'let': 1, 'wo..."
4,TRAAAEW128F42930C0,"{'i': 4, 'to': 5, 'and': 7, 'a': 2, 'me': 4, '...","{'like': 1, 'take': 1, 'would': 1, 'wo': 1, 's..."
...,...,...,...
210514,TRZZZWS128F429CF87,"{'a': 1, 'no': 9, 'que': 7, 'de': 1, 'y': 4, '...","{'que': 7, 'de': 1, 'en': 1, 'te': 5, 'tu': 4,..."
210515,TRZZZXA128F428ED56,"{'i': 1, 'the': 13, 'you': 6, 'to': 5, 'and': ...","{'time': 1, 'la': 1, 'get': 1, 'eye': 1, 'thin..."
210516,TRZZZXV128F4289747,"{'i': 13, 'the': 3, 'you': 17, 'to': 5, 'and':...","{'know': 1, 'like': 3, 'time': 1, 'get': 3, 'n..."
210517,TRZZZYV128F92E996D,"{'i': 10, 'the': 6, 'you': 20, 'and': 2, 'me':...","{'get': 21, 'got': 3, 'let': 6, 'would': 2, 'a..."


In [10]:
nltk.download('words')

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\mfard\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\words.zip.


True

In [11]:
progress = 1

def is_english_word(word_counts, total_rows):
    global progress
    print(f"Progress: {(progress * 100 / total_rows):.2f}% → working on {progress} of {total_rows}", end='\r', flush=True)
    progress += 1
    
    english_words = set(words.words())

    non_english_count = 0
    filtered_english_counts = {}
    for word, count in word_counts.items():        
        if word not in english_words:
            non_english_count += count
        else:
            filtered_english_counts[word] = count

    total = sum(word_counts.values())

    if non_english_count > total / 2:
        return None
    else:
        return filtered_english_counts

In [12]:
df['filtered_english'] = df['filtered_word_counts'].apply(is_english_word, total_rows=len(df))
df = df.dropna(subset=['filtered_english'])
df

,track_id,word_counts,filtered_word_counts,filtered_english
0,TRAAAAV128F421A322,"{'i': 6, 'the': 4, 'you': 2, 'to': 2, 'and': 5...","{'like': 2, 'de': 1, 'got': 1, 'would': 1, 'se...","{'like': 2, 'de': 1, 'got': 1, 'would': 1, 'se..."
1,TRAAABD128F429CF47,"{'i': 10, 'you': 17, 'to': 8, 'and': 2, 'a': 2...","{'know': 5, 'time': 3, 'la': 7, 'get': 2, 'got...","{'know': 5, 'time': 3, 'la': 7, 'get': 2, 'got..."
2,TRAAAED128E0783FAB,"{'i': 28, 'the': 15, 'you': 2, 'to': 12, 'and'...","{'love': 11, 'like': 1, 'time': 6, 'come': 4, ...","{'love': 11, 'like': 1, 'time': 6, 'come': 4, ..."
3,TRAAAEF128F4273421,"{'i': 5, 'the': 4, 'you': 3, 'to': 2, 'and': 1...","{'know': 1, 'got': 3, 'feel': 1, 'let': 1, 'wo...","{'know': 1, 'got': 3, 'feel': 1, 'let': 1, 'wo..."
4,TRAAAEW128F42930C0,"{'i': 4, 'to': 5, 'and': 7, 'a': 2, 'me': 4, '...","{'like': 1, 'take': 1, 'would': 1, 'wo': 1, 's...","{'like': 1, 'take': 1, 'would': 1, 'wo': 1, 's..."
...,...,...,...,...
210513,TRZZZUK128F92E3C60,"{'i': 8, 'the': 2, 'you': 10, 'to': 2, 'and': ...","{'love': 2, 'see': 2, 'heart': 2, 'gonna': 2, ...","{'love': 2, 'see': 2, 'heart': 2, 'think': 1, ..."
210515,TRZZZXA128F428ED56,"{'i': 1, 'the': 13, 'you': 6, 'to': 5, 'and': ...","{'time': 1, 'la': 1, 'get': 1, 'eye': 1, 'thin...","{'time': 1, 'la': 1, 'get': 1, 'eye': 1, 'thin..."
210516,TRZZZXV128F4289747,"{'i': 13, 'the': 3, 'you': 17, 'to': 5, 'and':...","{'know': 1, 'like': 3, 'time': 1, 'get': 3, 'n...","{'know': 1, 'like': 3, 'time': 1, 'get': 3, 'n..."
210517,TRZZZYV128F92E996D,"{'i': 10, 'the': 6, 'you': 20, 'and': 2, 'me':...","{'get': 21, 'got': 3, 'let': 6, 'would': 2, 'a...","{'get': 21, 'got': 3, 'let': 6, 'would': 2, 'a..."


In [13]:
emotions_list = pd.read_csv('data/emotion_lists.csv', header=0, converters={'emotions':ast.literal_eval})
emotions_list

,word,emotions
0,aback,[]
1,abacus,[]
2,abandon,"[fear, sad]"
3,abandoned,"[angry, fear, sad]"
4,abandonment,"[angry, fear, sad, surprise]"
...,...,...
14149,zone,[]
14150,zoo,[]
14151,zoological,[]
14152,zoology,[]


In [14]:
progress = 1

def emotion_map(word_counts, total_rows):
    global progress
    print(f"Progress: {(progress * 100 / total_rows):.2f}% → working on {progress} of {total_rows}", end='\r', flush=True)
    progress += 1

    emotions = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
    freqs = {emotion:0 for emotion in emotions}

    total = 0
    for word, count in word_counts.items():
        
        if word in emotions_list['word'].values:
            emotions = emotions_list.loc[emotions_list['word'] == word, 'emotions'].values[0]
            if not emotions:
                continue

            for emotion in emotions:
                freqs[emotion] += count
                total += count

    if total != 0:
        for emotion in freqs.keys():
            freqs[emotion] /= total

    return freqs


In [15]:
df['emotion_freqs'] = df['filtered_english'].apply(emotion_map, total_rows=len(df))

C:\Users\mfard\AppData\Local\Temp\ipykernel_25476\4246308584.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['emotion_freqs'] = df['filtered_english'].apply(emotion_map, total_rows=len(df))


In [16]:
final_df = df[['track_id', 'filtered_english', 'emotion_freqs']]
final_df = final_df.rename(columns={'filtered_english':'word_counts'})
final_df

,track_id,word_counts,emotion_freqs
0,TRAAAAV128F421A322,"{'like': 2, 'de': 1, 'got': 1, 'would': 1, 'se...","{'angry': 0.05, 'disgust': 0.15, 'fear': 0.15,..."
1,TRAAABD128F429CF47,"{'know': 5, 'time': 3, 'la': 7, 'get': 2, 'got...","{'angry': 0.0, 'disgust': 0.0, 'fear': 0.0, 'h..."
2,TRAAAED128E0783FAB,"{'love': 11, 'like': 1, 'time': 6, 'come': 4, ...","{'angry': 0.0, 'disgust': 0.0, 'fear': 0.02941..."
3,TRAAAEF128F4273421,"{'know': 1, 'got': 3, 'feel': 1, 'let': 1, 'wo...","{'angry': 0.07142857142857142, 'disgust': 0.07..."
4,TRAAAEW128F42930C0,"{'like': 1, 'take': 1, 'would': 1, 'wo': 1, 's...","{'angry': 0.125, 'disgust': 0.5, 'fear': 0.125..."
...,...,...,...
210513,TRZZZUK128F92E3C60,"{'love': 2, 'see': 2, 'heart': 2, 'think': 1, ...","{'angry': 0.0, 'disgust': 0.0, 'fear': 0.0, 'h..."
210515,TRZZZXA128F428ED56,"{'time': 1, 'la': 1, 'get': 1, 'eye': 1, 'thin...","{'angry': 0.14705882352941177, 'disgust': 0.14..."
210516,TRZZZXV128F4289747,"{'know': 1, 'like': 3, 'time': 1, 'get': 3, 'n...","{'angry': 0.6923076923076923, 'disgust': 0.076..."
210517,TRZZZYV128F92E996D,"{'get': 21, 'got': 3, 'let': 6, 'would': 2, 'a...","{'angry': 0.2624113475177305, 'disgust': 0.234..."


In [17]:
final_df.to_csv('data/training_data_full.csv', index=False)